In [2]:
import pandas as pd
import cantools

def parse_excel_spec(excel_file):
    """Parse the Excel specification file."""
    df = pd.read_excel(excel_file)  # Read the Excel file into a DataFrame
    messages = []  # List to store messages
    signals = {}   # Dictionary to store signals, keyed by message ID

    for index, row in df.iterrows():
        if row['Type'] == 'Message':
            try:
                # Convert hexadecimal string to integer for the CAN ID
                message_id = int(str(row['CAN ID']), 16)
            except ValueError:
                print(f"Skipping row {index + 1}: Invalid hexadecimal ID value '{row['CAN ID']}'")
                continue
            
            # Extract message details, with default values if columns are missing or NaN
            dlc = int(row['DLC']) if 'DLC' in df.columns and not pd.isna(row['DLC']) else 8
            cycle_time = int(row['Cycle Time']) if 'Cycle Time' in df.columns and not pd.isna(row['Cycle Time']) else None
            bms1 = row['BMS1'] if 'BMS1' in df.columns and not pd.isna(row['BMS1']) else ''
            bms2 = row['BMS2'] if 'BMS2' in df.columns and not pd.isna(row['BMS2']) else ''
            cluster = row['Cluster'] if 'Cluster' in df.columns and not pd.isna(row['Cluster']) else ''
            mcu = row['MCU'] if 'MCU' in df.columns and not pd.isna(row['MCU']) else ''
            obc = row['OBC'] if 'OBC' in df.columns and not pd.isna(row['OBC']) else ''
            tcu = row['TCU'] if 'TCU' in df.columns and not pd.isna(row['TCU']) else ''
            comment = row['Comments'] if 'Comments' in df.columns and not pd.isna(row['Comments']) else ''
            
            # Determine transmitters and receivers based on BMS1, BMS2, Cluster, MCU, OBC, and TCU values
            transmitters = []
            receivers = []
            if bms1 == 'Tx':
                transmitters.append('BMS1')
            if bms1 == 'Rx':
                receivers.append('BMS1')
            if bms2 == 'Tx':
                transmitters.append('BMS2')
            if bms2 == 'Rx':
                receivers.append('BMS2')
            if cluster == 'Tx':
                transmitters.append('Cluster')
            if cluster == 'Rx':
                receivers.append('Cluster')
            if mcu == 'Tx':
                transmitters.append('MCU')
            if mcu == 'Rx':
                receivers.append('MCU')
            if obc == 'Tx':
                transmitters.append('OBC')
            if obc == 'Rx':
                receivers.append('OBC')
            if tcu == 'Tx':
                transmitters.append('TCU')
            if tcu == 'Rx':
                receivers.append('TCU')

            # Append the message details to the messages list
            messages.append({
                'message_id': message_id,
                'name': row['Message/Signal Name'],
                'type': row['Message Type'],
                'dlc': dlc,
                'cycle_time': cycle_time,
                'transmitters': transmitters,
                'receivers': receivers,
                'comment': comment
            })
        
        elif row['Type'] == 'Signal':
            try:
                # Convert hexadecimal string to integer for the Message ID
                message_id = int(str(row['Message ID']), 16)
            except ValueError:
                print(f"Skipping row {index + 1}: Invalid hexadecimal Message ID value '{row['Message ID']}'")
                continue
            
            # Extract signal values as a dictionary if present
            signal_values = {}
            if 'Values' in df.columns and not pd.isna(row['Values']):
                values_str = row['Values']
                values_pairs = values_str.split(', ')
                for pair in values_pairs:
                    try:
                        key, val = pair.split('=')
                        signal_values[int(key.strip())] = val.strip().strip('"')
                    except ValueError:
                        print(f"Skipping invalid value pair '{pair}' in row {index + 1}")
                        continue
            
            # Append the signal details to the signals dictionary
            signals.setdefault(message_id, []).append({
                'name': row['Message/Signal Name'],
                'start_bit': int(row['Start Bit']),
                'length': int(row['Length']),
                'is_little_endian': bool(row['Little Endian']),
                'is_signed': bool(row['Signed']),
                'scale': float(row['Factor']),
                'offset': float(row['Offset']),
                'minimum': float(row['Minimum']) if 'Minimum' in df.columns and not pd.isna(row['Minimum']) else None,
                'maximum': float(row['Maximum']) if 'Maximum' in df.columns and not pd.isna(row['Maximum']) else None,
                'unit': row['Unit'] if 'Unit' in df.columns and not pd.isna(row['Unit']) else None,
                'values': signal_values,  # Assign value enumerations
                'comment': row['Comments'] if 'Comments' in df.columns and not pd.isna(row['Comments']) else ''
            })

    return messages, signals

def calculate_dlc(signals):
    """Calculate the Data Length Code (DLC) based on the signals."""
    max_start_bit = max(signal['start_bit'] + signal['length'] for signal in signals)
    return (max_start_bit + 7) // 8  # Calculate the required number of bytes

def create_dbc(messages, signals):
    """Create a CAN database object using cantools."""
    message_definitions = []
    
    # Add messages and signals to the list
    for message_data in messages:
        message_id = message_data['message_id']
        dlc = message_data['dlc']  # Use provided DLC
        if dlc is None:  # If DLC is not provided, calculate it
            dlc = calculate_dlc(signals.get(message_id, []))
        
        # Create a CAN message definition
        message = cantools.db.Message(
            frame_id=message_id,
            name=message_data['name'],
            length=dlc,
            senders=message_data['transmitters'],  # List of transmitters
            is_extended_frame=False,  # Modify if needed
            cycle_time=message_data['cycle_time'],  # Cycle time if available
            signals=[],  # Provide an empty list for signals
            comment=message_data['comment'],  # Add comments
            bus_name=None,  # Add bus name if needed
            strict=True  # Strict checking of the signal start bits and lengths
        )
        
        # Add signals to the message
        for signal_data in signals.get(message_id, []):
            signal = cantools.db.Signal(
                name=signal_data['name'],
                start=signal_data['start_bit'],
                length=signal_data['length'],
                byte_order='little_endian' if signal_data['is_little_endian'] else 'big_endian',
                is_signed=signal_data['is_signed'],  # Use the new 'is_signed' field
                receivers=message_data['receivers'],  # Add list of receivers
                minimum=signal_data['minimum'],  # Add minimum value
                maximum=signal_data['maximum'],  # Add maximum value
                unit=signal_data['unit'],  # Add unit
                comment=signal_data['comment'],  # Add comments
            )
            # Set additional signal properties
            signal.scale = signal_data['scale']  # Add scale factor to signal
            signal.offset = signal_data['offset']  # Add offset to signal
            signal.choices = signal_data['values']  # Add value enumerations
            message.signals.append(signal)
        
        message_definitions.append(message)

    # Create the CAN database with the message definitions
    db = cantools.db.Database(messages=message_definitions)
    return db

def save_dbc_file(db, dbc_file):
    """Save the CAN database to a .dbc file."""
    with open(dbc_file, 'w') as f:
        f.write(db.as_dbc_string())

if __name__ == "__main__":
    excel_file = r"D:\UltraLite\Ultra_Lite.xlsx"
    dbc_file = 'UltraLite.dbc'

    # Step 1: Parse the Excel specification file
    messages, signals = parse_excel_spec(excel_file)

    # Step 2: Create a CAN database object
    db = create_dbc(messages, signals)

    # Step 3: Save the CAN database to a .dbc file
    save_dbc_file(db, dbc_file)

    print(f".dbc file created: {dbc_file}")


ValueError: could not convert string to float: '###########'